In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain

ProjDir = rel_path("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col logpop, set log() for population data

In [2]:
d[:logpop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6_1 = "
data{
    int total_tools[10];
    real logpop[10];
    int society[10];
}
parameters{
    real a;
    real bp;
    vector[10] a_society;
    real<lower=0> sigma_society;
}
model{
    vector[10] mu;
    sigma_society ~ cauchy( 0 , 1 );
    a_society ~ normal( 0 , sigma_society );
    bp ~ normal( 0 , 1 );
    a ~ normal( 0 , 10 );
    for ( i in 1:10 ) {
        mu[i] = a + a_society[society[i]] + bp * logpop[i];
        mu[i] = exp(mu[i]);
    }
    total_tools ~ poisson( mu );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6.1",  model=m12_6_1, num_warmup=2000,
num_samples=2000, output_format=:mcmcchain);

Input data for cmdstan

In [4]:
m12_6_1_data = Dict("total_tools" => d[:total_tools], "logpop" => d[:logpop], "society" => d[:society]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_1_data, ProjDir, diagnostics=false, CmdStanDir=CMDSTAN_HOME);

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.1.stan' at line 15)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.


Inference for Stan model: m12_6_1_model
4 chains: each with iter=(2000,2000,2000,2000); warmup=(0,0,0,0); thin=(1,1,1,1); 8000 iterations saved.

Warmup took (0.94, 0.89, 0.91, 0.96) seconds, 3.7 seconds total
Sampling took (1.1, 1.0, 1.1, 1.2) seconds, 4.5 seconds total

                    Mean     MCSE   StdDev        5%       50%       95%    N_Eff  N_Eff/s    R_hat
lp__             9.3e+02  8.1e-02  3.2e+00   9.2e+02   9.3e+02   9.3e+02  1.5e+03  3.4e+02  1.0e+00
accept_stat__    

Describe the draws

In [6]:
describe(chn)

Iterations = 1:2000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 2000

Empirical Posterior Estimates:
                   Mean            SD         Naive SE         MCSE         ESS   
         lp__  927.812855375  3.1703286917 0.035445352329 0.08199076746 1495.12905
accept_stat__    0.948476282  0.0739979129 0.000827321818 0.00150535109 2000.00000
   stepsize__    0.046153400  0.0047941735 0.000053600489 0.00053935294   79.00988
  treedepth__    5.964625000  0.4595106126 0.005137484831 0.01311939302 1226.77130
 n_leapfrog__   78.944500000 39.5183265336 0.441828322431 1.11342907301 1259.71406
  divergent__    0.000000000  0.0000000000 0.000000000000 0.00000000000        NaN
     energy__ -921.254060750  4.0873302303 0.045697741208 0.09953322907 1686.33272
            a    1.109204913  0.7784442794 0.008703271627 0.01672073204 2000.00000
           bp    0.260064330  0.0840200665 0.000939372901 0.00178576423 2000.00000
  a_society.1   -0.203252873  0.2466843932 0.002758015

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*